![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

# 1. Import Libraries

In [1]:
# Import libraries to use
import pandas as pd
import numpy as np

# 2. Import Dataset

In [2]:
# Import dataset
df = pd.read_csv('bank_marketing.csv', delimiter = ',') 
print(df.isna().info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   client_id       41188 non-null  bool 
 1   age             41188 non-null  bool 
 2   job             41188 non-null  bool 
 3   marital         41188 non-null  bool 
 4   education       41188 non-null  bool 
 5   credit_default  41188 non-null  bool 
 6   housing         41188 non-null  bool 
 7   loan            41188 non-null  bool 
 8   contact         41188 non-null  bool 
 9   month           41188 non-null  bool 
 10  day             41188 non-null  bool 
 11  duration        41188 non-null  bool 
 12  campaign        41188 non-null  bool 
 13  pdays           41188 non-null  bool 
 14  previous        41188 non-null  bool 
 15  poutcome        41188 non-null  bool 
 16  emp_var_rate    41188 non-null  bool 
 17  cons_price_idx  41188 non-null  bool 
 18  cons_conf_idx   41188 non-

# 3. Reading in and Splitting the data

In [3]:
# Create new dataframe for Client
client_df = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]

# Create new dataframe for Campaign
campaign_df = df[['client_id','campaign', 'duration', 'pdays', 'previous', 'poutcome', 'y', 'day', 'month']]

# Create new dataframe for Economics
economics_df = df[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m',  'nr_employed']]

# 4. Data Preparation for each Dataframe

## 4.1. Dataframe of Client

In [4]:
# Check all columns in Dataframe for Client
print(client_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       41188 non-null  object
 5   credit_default  41188 non-null  object
 6   housing         41188 non-null  object
 7   loan            41188 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.5+ MB
None


In [5]:
# Rename several columns to met the requirements
client_df.rename(columns={"client_id": "id"}, inplace=True)

print(client_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       41188 non-null  object
 5   credit_default  41188 non-null  object
 6   housing         41188 non-null  object
 7   loan            41188 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.5+ MB
None


In [6]:
# Clean the "education" column, changing "." to "_" and "unknown" to NumPy's null values.
client_df['education'] = client_df['education'].str.replace('.','_')
client_df['education'] = client_df['education'].replace('unknow', np.NaN)

print(client_df[['education']])

                 education
0                 basic_4y
1              high_school
2              high_school
3                 basic_6y
4              high_school
...                    ...
41183  professional_course
41184  professional_course
41185    university_degree
41186  professional_course
41187  professional_course

[41188 rows x 1 columns]


In [7]:
# Remove periods from the "job" column.
client_df['job'] = client_df['job'].str.replace('.','')

# Check again
print(client_df[['job']].info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   job     41188 non-null  object
dtypes: object(1)
memory usage: 321.9+ KB
None


## 4.2. Dataframe of Campaign

In [8]:
# Rename several columns to met the requirements
campaign_df.rename(columns={'duration':'contact_duration',
 'previous': 'previous_campaign_contacts',
 'y':'campaign_outcome',
 'poutcome': 'previous_outcome',
 'campaign': 'number_contacts',
}, inplace=True)

print(campaign_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  int64 
 1   number_contacts             41188 non-null  int64 
 2   contact_duration            41188 non-null  int64 
 3   pdays                       41188 non-null  int64 
 4   previous_campaign_contacts  41188 non-null  int64 
 5   previous_outcome            41188 non-null  object
 6   campaign_outcome            41188 non-null  object
 7   day                         41188 non-null  int64 
 8   month                       41188 non-null  object
dtypes: int64(6), object(3)
memory usage: 2.8+ MB
None


In [9]:
# Check the column names of the DataFrame
campaign_df['campaign_outcome'] = campaign_df['campaign_outcome'].map({'yes': 1, 'no': 0})

# Assuming 'campaign_outcome' is a valid column name, print it
if 'campaign_outcome' in campaign_df.columns:
    print(campaign_df['campaign_outcome'])
else:
    print("The column 'campaign_outcome' does not exist in the DataFrame.")

0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: campaign_outcome, Length: 41188, dtype: int64


In [10]:
# Convert "success" and "failure" in the "previous_outcome" and "campaign_outcome" columns to binary (1 or 0), along with the changing "nonexistent" to NumPy's null values in "previous_outcome".
campaign_df['previous_outcome'] = campaign_df['previous_outcome'].replace('nonexistent', np.NaN)

# Assuming 'previous_outcome' is a valid column name, you can print it
if 'previous_outcome' in campaign_df.columns:
    print(campaign_df['previous_outcome'])
else:
    print("The column 'previous_outcome' does not exist in the DataFrame.")
    


0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
41183        NaN
41184        NaN
41185        NaN
41186        NaN
41187    failure
Name: previous_outcome, Length: 41188, dtype: object


In [11]:
# Add a column called campaign_id in campaign, where all rows have a value of 1. 
campaign_df['campaign_id'] = 1
print(campaign_df)

       client_id  number_contacts  contact_duration  ...  day  month campaign_id
0              0                1               261  ...   13    may           1
1              1                1               149  ...   19    may           1
2              2                1               226  ...   23    may           1
3              3                1               151  ...   27    may           1
4              4                1               307  ...    3    may           1
...          ...              ...               ...  ...  ...    ...         ...
41183      41183                1               334  ...   30    nov           1
41184      41184                1               383  ...    6    nov           1
41185      41185                2               189  ...   24    nov           1
41186      41186                1               442  ...   17    nov           1
41187      41187                3               239  ...   23    nov           1

[41188 rows x 10 columns]


In [12]:
# Create a datetime column called last_contact_date, in the format of "year-month-day", where the year is 2022, and the month and day values are taken from the "month" and "day" columns.

campaign_df['month'] = campaign_df['month'].str.capitalize()

# Add new column 'year' which has value '2022' for all rows.
campaign_df['year'] = '2022'

# Covert data type of column 'day' to string
campaign_df['day'] = campaign_df['day'].astype(str)

# Create a datetime column called last_contact_date
campaign_df['last_contact_date'] = campaign_df['year'] + '-' + campaign_df['month'] + '-' + campaign_df['day']

# Convert data type of 'last_contact_date' to datetime
campaign_df['last_contact_date'] = pd.to_datetime(campaign_df['last_contact_date'], format='%Y-%b-%d')

print(campaign_df['last_contact_date'])


0       2022-05-13
1       2022-05-19
2       2022-05-23
3       2022-05-27
4       2022-05-03
           ...    
41183   2022-11-30
41184   2022-11-06
41185   2022-11-24
41186   2022-11-17
41187   2022-11-23
Name: last_contact_date, Length: 41188, dtype: datetime64[ns]


In [13]:
## Remove any redundant data
campaign_df = campaign_df.drop(['day','month','year'], axis = 1)

print(campaign_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   pdays                       41188 non-null  int64         
 4   previous_campaign_contacts  41188 non-null  int64         
 5   previous_outcome            5625 non-null   object        
 6   campaign_outcome            41188 non-null  int64         
 7   campaign_id                 41188 non-null  int64         
 8   last_contact_date           41188 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(7), object(1)
memory usage: 2.8+ MB
None


## 4.3. Dataframe of Economic

In [14]:
# Rename several columns to met the requirements
economics_df.rename(columns={'euribor3m':'euribor_three_months',
 'nr_employed': 'number_employed'}, inplace=True)

print(economics_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   emp_var_rate          41188 non-null  float64
 2   cons_price_idx        41188 non-null  float64
 3   euribor_three_months  41188 non-null  float64
 4   number_employed       41188 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 1.6 MB
None


# 5. Export to CSV files

In [15]:
# Export 3 dataframes into 3 csv files
client_df.to_csv('client.csv', index = False)
campaign_df.to_csv('campaign.csv', index = False)
economics_df.to_csv('economics.csv', index = False)

# 6. Designing the databse

In [16]:
# Table for Client
client_table = """
CREATE TABLE client (
    id SERIAL PRIMARY KEY,
    age INT,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOL,
    housing BOOL,
    loan BOOL
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

# Table for Campaign
campaign_table = """
CREATE TABLE campaign (
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL REFERENCES client (id),
    number_contacts INT,
    contact_duration INT,
    pdays INT,
    previous_campaign_contacts INT,
    previous_outcome BOOL,
    campaign_outcome BOOL,
    last_contact_date date
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

# Table for Economics
economics_table = """
CREATE TABLE economics (
    client_id SERIAL REFERENCES client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""